In [236]:
import sys
import yaml
import itertools
import subprocess

In [237]:
from pathlib import Path

# Compile original Lyra2 implementations

Need to import python functions that compile Lyra from a git submodule. Manipulate path as per https://stackoverflow.com/a/29747054/1269892

In [238]:
sys.path.append(str(Path('../lyra/Lyra2/tests').resolve()))

In [239]:
from harness import build_lyra2, unlist_values
from harness import compose_sponge_name, compose_lyra2_name

In [240]:
with open('harness.yml', 'r') as config:
    params = yaml.load(config)

In [241]:
build_lyra2(params)

In [242]:
build_path0 = Path('./bin42').resolve()
if not build_path0.exists():
    print('Could not find ./bin42, directory for original executables')

# Compile ported Java implementation

In [243]:
subprocess.run(['mvn', 'package', '-f', '../lyra2-java', '-Plyra2-compare', '-DskipTests'])

CompletedProcess(args=['mvn', 'package', '-f', '../lyra2-java', '-Plyra2-compare', '-DskipTests'], returncode=0)

In [244]:
build_path1 = Path('./target').resolve()
if not build_path1.exists():
    print('Could not ./target, directory for compiled executables')

# Class to measure time and memory usage

Resources used for memory measurements:

https://stackoverflow.com/questions/22372960/is-this-explanation-about-vss-rss-pss-uss-accurate

In [245]:
import time
import psutil

In [246]:
from statistics import median

In [247]:
class ProcessObserver:
    def __init__(self, ntimes=3, mtimes=3):
        # Number of times to measure elapsed time
        self.ntimes = ntimes
        # Number of times to measure consumed memory
        self.mtimes = mtimes

    def run(self, *args):
        # Run the process self.ntimes and see how long it takes
        times = [-1 for i in range(self.ntimes)]
        for i in range(self.ntimes):
            fst = time.time()
            
            process = subprocess.run(*args)
                        
            lst = time.time()
            
            if process.returncode != 0:
                print(args[0] + ' failed to run')

                continue
            
            times[i] = lst - fst
            
        mtime = median(times)
        
        process = subprocess.Popen(*args)
        
        # Approximate running time of the process is known
        # The process has just been started asynchronously
        # Measure its memory usage (which is complicated)
        pss_mems = [-1 for i in range(self.mtimes)]
        uss_mems = [-1 for i in range(self.mtimes)]
        for i in range(self.mtimes):
            if process.poll() is not None:
                break # the process no longer runs, break

            # the process will finish when you least expect it
            # so wrap everything into try-except and handle it
            try:
                p = psutil.Process(process.pid)
                mem = p.memory_full_info()

                # total private memory + proporional size for the 3pp libraries
                pss_mems[i] = mem.pss
                # total private memory of a process (unique to the process)
                uss_mems[i] = mem.uss
            except: 
                break # the process no longer runs, break

            time.sleep(mtime / self.mtimes)
            
            
        return [median(times), max(pss_mems), max(uss_mems)]

# Perform the measurements

In [248]:
password, salt = 'password', 'salt'

In [249]:
klens = [1, 10, 100]

In [250]:
tcosts = [1, 10, 100]

In [251]:
mcosts = [3, 10, 100] # there must be at least 3 rows in the memory matrix

In [252]:
observer = ProcessObserver(ntimes=3, mtimes=5)

In [253]:
time_data0 = {}
time_data1 = {}

# Java implementataion accepts sponge type, number of rounds etc as runtime parameters
# which is why the name of the executable does not change in contrast to original
executable1 = build_path1.joinpath('lyra2-1.2-SNAPSHOT-jar-with-dependencies.jar')

for matrix in unlist_values(params['matrix']):
    option = matrix['option']

    threads = matrix['threads']
    columns = matrix['columns']

    sponge = matrix['sponge']
    rounds = matrix['rounds']
    blocks = matrix['blocks']

    bench = matrix['bench']

    [sponge, _] = compose_sponge_name(sponge)

    name = compose_lyra2_name(
        option, threads, columns, sponge, rounds, blocks
    )
    
    executable0 = build_path0.joinpath(name)    
    
    time_data0[name] = {}
    time_data1[name] = {}
    
    for klen, tcost, mcost in itertools.product(klens, tcosts, mcosts):
        
        time_data0[name][klen, tcost, mcost] = observer.run([
            executable0
            , password
            , salt
            , str(klen)
            , str(tcost)
            , str(mcost)
        ])
        
        time_data1[name][klen, tcost, mcost] = observer.run([
            'java'
            , '-jar'         , str(executable1)
            , '--blocks'     , str(blocks)
            , '--columns'    , str(columns)
            , '--full-rounds', str(rounds)
            , '--half-rounds', str(rounds)
            , '--sponge'     , sponge
            , password
            , salt
            , str(klen)
            , str(tcost)
            , str(mcost)
        ])

In [254]:
print(time_data0)

{'lyra2-generic-x86-64-threads-1-columns-256-sponge-blake2b-rounds-1-blocks-8': {(1, 1, 3): [0.0023784637451171875, 178176, 139264], (1, 1, 10): [0.0023124217987060547, 281600, 241664], (1, 1, 100): [0.0038335323333740234, 1821696, 1785856], (1, 10, 3): [0.0023217201232910156, 182272, 143360], (1, 10, 10): [0.002820253372192383, 334848, 299008], (1, 10, 100): [0.009460687637329102, 1819648, 1785856], (1, 100, 3): [0.004198312759399414, 230400, 188416], (1, 100, 10): [0.008324623107910156, 347136, 311296], (1, 100, 100): [0.06583452224731445, 1825792, 1785856], (10, 1, 3): [0.0023736953735351562, 178176, 143360], (10, 1, 10): [0.0027043819427490234, 265216, 229376], (10, 1, 100): [0.0038983821868896484, 1825792, 1785856], (10, 10, 3): [0.003268003463745117, 187392, 151552], (10, 10, 10): [0.0028243064880371094, 336896, 303104], (10, 10, 100): [0.009592771530151367, 1825792, 1785856], (10, 100, 3): [0.004096269607543945, 229376, 188416], (10, 100, 10): [0.009565353393554688, 352256, 3112

In [255]:
print(time_data1)

{'lyra2-generic-x86-64-threads-1-columns-256-sponge-blake2b-rounds-1-blocks-8': {(1, 1, 3): [0.09749150276184082, 26006528, 25931776], (1, 1, 10): [0.09621047973632812, 25903104, 25825280], (1, 1, 100): [0.14840960502624512, 28310528, 28233728], (1, 10, 3): [0.11756038665771484, 26899456, 26824704], (1, 10, 10): [0.12423467636108398, 27435008, 27365376], (1, 10, 100): [0.2825791835784912, 29649920, 29585408], (1, 100, 3): [0.1739211082458496, 27957248, 27885568], (1, 100, 10): [0.27100563049316406, 28127232, 28061696], (1, 100, 100): [1.6378390789031982, 33502208, 33435648], (10, 1, 3): [0.09101653099060059, 25732096, 25665536], (10, 1, 10): [0.09831881523132324, 26327040, 26247168], (10, 1, 100): [0.1426715850830078, 28695552, 28626944], (10, 10, 3): [0.11502289772033691, 27052032, 26972160], (10, 10, 10): [0.13275957107543945, 27461632, 27381760], (10, 10, 100): [0.2934386730194092, 29809664, 29728768], (10, 100, 3): [0.174269437789917, 26490880, 26419200], (10, 100, 10): [0.29614639